In [1]:
import numpy as np
import pde
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.optimize import curve_fit
import plotly.express as px
from matplotlib.colors import ListedColormap

import clevercsv
import glob
import os
import pandas as pd
import matplotlib.cm as cm
from os.path import exists

import plotly.express as px
import plotly.graph_objects as go
import scipy


In [2]:
def sim_numerical_growth_diffusion(u0, g_x, L, T_cycle, num_T, beta_u, beta_g=0, bc_x = ({"value": 0}, {"value": 0}), IsPeriodic=False, IsPlot=False):
    expr = {'u' : f'laplace(u) * {beta_u} + g * u', 
        'g' : f'{beta_g} * laplace(g)'}
    
    if IsPeriodic:
        eq = pde.PDE(expr)
    else:    
        eq = pde.PDE(expr, bc=[bc_x, ])
    grid = pde.UnitGrid([L,], periodic=IsPeriodic) # unit grid, for each well
    
    ## D: drug concentration, LZD gives us binary rsult for gs that's why our experiment works.
    # expr = {'u' : f'laplace(u) * {beta} + g0/(1+(D/K)**n) * u', 
    #         'D' : f'{beta_D} * laplace(D)'}

    # Construct the field objects
    initial_u = pde.ScalarField(grid, data=u0)
    initial_g = pde.ScalarField(grid, data=g_x)
    # Set the initial values for all fields, and show it
    initial_state = pde.FieldCollection([initial_u,initial_g])
   

    # simulate the dynamics
    storage = pde.MemoryStorage()
    final_state = eq.solve(state=initial_state, 
                           t_range=num_T*T_cycle, 
                           dt=T_cycle, 
                           tracker=['progress', storage.tracker(T_cycle)],
                           solver="scipy", # DEFAULT: solver: str | SolverBase = "explicit"
                           ret_info = False,)   
    if IsPlot:
      initial_state.plot(title='Initial state')
      final_state.plot()
    
    return storage


## an example for homogeneous drug environment

## an example for heterogeneous drug environment